# SLOTS

In [44]:
class SlotsClass:
    __slots__ = ['name', 'age', 'age1', 'age2', 
                 'age3', 'age4']
    def __init__(self):
        self.name = 'John'
        self.age = 20
        self.age1 = 20
        self.age2 = 20
        self.age3 = 20
        self.age4 = 20

        
class StandartClass:
    def __init__(self):
        self.name = 'John'
        self.age = 20
        self.age1 = 20
        self.age2 = 20
        self.age3 = 20
        self.age4 = 20


In [45]:
sloted = SlotsClass()
standart = StandartClass()

## Memory usage

In [48]:
from sys import getsizeof
getsizeof(sloted), getsizeof(standart.__dict__)

(88, 152)

## Speed access

In [16]:
import timeit

class Foo(object): __slots__ = 'foo',

class Bar(object): pass

slotted = Foo()
not_slotted = Bar()

def get_set_delete_fn(obj):
    def get_set_delete():
        obj.foo = 'foo'
        obj.foo
        del obj.foo
    return get_set_delete


sl = min(timeit.repeat(get_set_delete_fn(slotted)))
nsl = min(timeit.repeat(get_set_delete_fn(not_slotted)))
speed_increese = round(nsl / sl * 100, 0) - 100
print(f"Slotted accesse is almost {speed_increese}% faster.")

Slotted accesse is almost 29.0% faster.


#### TLDR:

The special attribute \_\_slots__ allows you to explicitly state which instance attributes you expect your object instances to have, with the expected results:

1. faster attribute access.
2. space savings in memory.

#### The space savings is from

1. Storing value references in slots instead of __dict__.
2. enying __dict__ and __weakref__ creation if parent classes deny them and you declare __slots__.


You should only declare a particular slot one time in an inheritance tree. For example:

In [1]:
class Base:
    __slots__ = 'foo', 'bar'

class Right(Base):
    __slots__ = 'baz', 

class Wrong(Base):
    __slots__ = 'foo', 'bar', 'baz' 

#### The biggest caveat is for multiple inheritance - multiple "parent classes with nonempty slots" cannot be combined.

In [20]:
class First:
    __slots__ = 'foo', 'bar'
    def __init__(self):
        self.foo = 1
        self.bar = 2
        
class Second:
    __slots__ = 'baz',
    def __init__(self):
        self.baz = 1
    
class Wrong(First, Second):
    __slots__ = 'foo', 'bar', 'baz' 


TypeError: multiple bases have instance lay-out conflict

To accommodate this restriction, follow best practices: Factor out all but one or all parents' abstraction which their concrete class respectively and your new concrete class collectively will inherit from - giving the abstraction(s) empty slots (just like abstract base classes in the standard library).

### Requirements:
* To have attributes named in \_\_slots__ to actually be stored in slots instead of a \_\_dict__, a class must inherit from object.
* To prevent the creation of a \_\_dict__, you must inherit from object and all classes in the inheritance must declare \_\_slots__ and none of them can have a '\_\_dict__' entry.

### Other cases to avoid slots:
* Avoid them when you want to perform \_\_class__ assignment with another class that doesn't have them (and you can't add them) unless the slot layouts are identical. (I am very interested in learning who is doing this and why.)
* Avoid them if you want to subclass variable length builtins like long, tuple, or str, and you want to add attributes to them.
* Avoid them if you insist on providing default values via class attributes for instance variables.
